### Exercise 3
Create an application that would examine the format of the films and convert them to the proper format if necessary

In [1]:
import subprocess
import os
import shutil
import json
import imageio_ffmpeg as ffmpeg

In [2]:
# Download a static FFmpeg build and add it to PATH.
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path
print('')
!which ffmpeg
print('Done!')


Done!


'which' is not recognized as an internal or external command,
operable program or batch file.


Check if ffmpeg is properly installed and what version is being used.

In [3]:
ffmpeg_path = ffmpeg.get_ffmpeg_version()

if ffmpeg_path:
    print(f"FFmpeg is installed at: {ffmpeg_path}")
else:
    print("FFmpeg not found. Please make sure it's installed.")

FFmpeg is installed at: 4.2.2


Check if ffmpeg and ffprobe is ready to be used and that it is placed in the system PATH.

In [4]:
ffmpeg_path = shutil.which('ffmpeg')
ffprobe_path = shutil.which('ffprobe')

if ffmpeg_path:
    print(f"FFmpeg is installed at: {ffmpeg_path}")
else:
    print("FFmpeg not found. Please make sure it's installed and added to the system PATH.")

if ffprobe_path:
    print(f"FFprobe is installed at: {ffprobe_path}")
else:
    print("FFprobe not found. Please make sure it's installed and added to the system PATH.")


FFmpeg is installed at: C:\ffmpeg\bin\ffmpeg.EXE
FFprobe is installed at: C:\ffmpeg\bin\ffprobe.EXE


Construct an FFprobe command to extract format and stream information in JSON format for all the videos in the "Exercise3_Films" folder.

In [5]:
# Specify the directory containing the video files
video_directory = "Exercise3_Films"

# Define a list of video file extensions to consider
video_extensions = [".mp4", ".avi", ".mov"]

# Iterate through each file in the directory
for filename in os.listdir(video_directory):
    if any(filename.lower().endswith(ext) for ext in video_extensions):
        video_file_path = os.path.join(video_directory, filename)

        # Run ffprobe command for the current video file
        ffprobe_command = f'ffprobe -v error -show_entries format:stream -of json "{video_file_path}"'

        try:
            ffprobe_output = subprocess.check_output(ffprobe_command, shell=True)
            # Parse the JSON output
            video_info = json.loads(ffprobe_output)
            
            print(f"Video Information for {filename}:")
            print(json.dumps(video_info, indent=2))
        except subprocess.CalledProcessError as e:
            print(f"Error running ffprobe for {filename}: {e}")

Video Information for Cosmos_War_of_the_Planets.mp4:
{
  "programs": [],
  "streams": [
    {
      "index": 0,
      "codec_name": "h264",
      "codec_long_name": "H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10",
      "profile": "Main",
      "codec_type": "video",
      "codec_tag_string": "avc1",
      "codec_tag": "0x31637661",
      "width": 628,
      "height": 354,
      "coded_width": 628,
      "coded_height": 354,
      "closed_captions": 0,
      "film_grain": 0,
      "has_b_frames": 0,
      "sample_aspect_ratio": "1:1",
      "display_aspect_ratio": "314:177",
      "pix_fmt": "yuv420p",
      "level": 31,
      "chroma_location": "left",
      "field_order": "progressive",
      "refs": 1,
      "is_avc": "true",
      "nal_length_size": "4",
      "id": "0x1",
      "r_frame_rate": "30000/1001",
      "avg_frame_rate": "30000/1001",
      "time_base": "1/30000",
      "start_pts": 0,
      "start_time": "0.000000",
      "duration_ts": 600600,
      "duration": "20.020000"

Grab only part of the format that is needed to be seen to check if any of the video files have any wrong format.

In [6]:
# Iterate through each file in the directory
for filename in os.listdir(video_directory):
    if any(filename.lower().endswith(ext) for ext in video_extensions):
        video_file_path = os.path.join(video_directory, filename)

        # Run ffprobe command for the current video file
        ffprobe_command = f'ffprobe -v error -show_entries format:stream -of json "{video_file_path}"'

        try:
            ffprobe_output = subprocess.check_output(ffprobe_command, shell=True)
            # Parse the JSON output
            video_info = json.loads(ffprobe_output)
            # Extract video format information
            video_format = video_info.get("format", {}).get("format_name")
            
            # Print the video format information
            print(f"Video Format for {filename}: {video_format}")
            
            # Find and print the codec_name and audio codec
            for stream in video_info.get("streams", []):
                if stream.get("index") == 0:
                    video_codec = stream.get("codec_name")
                    print(f"Video Codec for {filename}: {video_codec}")
                if stream.get("index") == 1:
                    audio_codec = stream.get("codec_name")
                    print(f"Audio Codec for {filename}: {audio_codec}")
            
            # Find and print the frame rate, aspect ratio, resolution, video bit rate, audio bit rate and audio channel
            for stream in video_info.get("streams", []):
                if stream.get("index") == 0:
                    avg_frame_rate = stream.get("avg_frame_rate")
                    aspect_ratio = stream.get("display_aspect_ratio")
                    width = stream.get("width")
                    height = stream.get("height")
                    video_bit_rate = stream.get("bit_rate")
                    
                    # Convert bit_rate to Mb/s
                    bit_rate_mbps = round(int(video_bit_rate)/1000000, 2)
                    
                    # Convert avg_frame_rate to decimal number
                    numerator, denominator = map(int, avg_frame_rate.split('/'))
                    frame_rate = round(numerator / denominator, 2)
                    
                    print(f"Frame rate for {filename}: {frame_rate} FPS")
                    print(f"Aspect Ratio for {filename}: {aspect_ratio}")
                    # Combine width and height to get resolution
                    print(f"Resolution for {filename}: {width} x {height}")
                    print(f"Video Bit Rate for {filename}: {bit_rate_mbps} Mb/s")
                    
                
                if stream.get("index") == 1:
                    audio_bit_rate = stream.get("bit_rate")
                    audio_channels = stream.get("channel_layout")
                    
                    # Convert audio bit rate into kpbs
                    bit_rate_kbps = round(int(audio_bit_rate)/1000, 2)
                    
                    print(f"Audio Bit Rate for {filename}: {bit_rate_kbps} kb/s")
                    print(f"Audio Channels for {filename}: {audio_channels}")
                    print()

        except subprocess.CalledProcessError as e:
            print(f"Error running ffprobe for {filename}: {e}")

Video Format for Cosmos_War_of_the_Planets.mp4: mov,mp4,m4a,3gp,3g2,mj2
Video Codec for Cosmos_War_of_the_Planets.mp4: h264
Audio Codec for Cosmos_War_of_the_Planets.mp4: aac
Frame rate for Cosmos_War_of_the_Planets.mp4: 29.97 FPS
Aspect Ratio for Cosmos_War_of_the_Planets.mp4: 314:177
Resolution for Cosmos_War_of_the_Planets.mp4: 628 x 354
Video Bit Rate for Cosmos_War_of_the_Planets.mp4: 2.99 Mb/s
Audio Bit Rate for Cosmos_War_of_the_Planets.mp4: 317.1 kb/s
Audio Channels for Cosmos_War_of_the_Planets.mp4: stereo

Video Format for Last_man_on_earth_1964.mov: mov,mp4,m4a,3gp,3g2,mj2
Video Codec for Last_man_on_earth_1964.mov: prores
Audio Codec for Last_man_on_earth_1964.mov: pcm_s16le
Frame rate for Last_man_on_earth_1964.mov: 23.98 FPS
Aspect Ratio for Last_man_on_earth_1964.mov: 16:9
Resolution for Last_man_on_earth_1964.mov: 640 x 360
Video Bit Rate for Last_man_on_earth_1964.mov: 9.29 Mb/s
Audio Bit Rate for Last_man_on_earth_1964.mov: 1536.0 kb/s
Audio Channels for Last_man_on_e

Generate a report showing all the video file's format and stating which format is not according to the proper format.

In [7]:
# Open the file in write mode
with open("report.txt", "w") as report_file:
    # Iterate through each file in the directory
    for filename in os.listdir(video_directory):
        if any(filename.lower().endswith(ext) for ext in video_extensions):
            video_file_path = os.path.join(video_directory, filename)

            # Run ffprobe command for the current video file
            ffprobe_command = f'ffprobe -v error -show_entries format:stream -of json "{video_file_path}"'

            try:
                ffprobe_output = subprocess.check_output(ffprobe_command, shell=True)
                # Parse the JSON output
                video_info = json.loads(ffprobe_output)
                # Extract video format information
                video_format = video_info.get("format", {}).get("format_name")

                # Check if the video format contains "mp4"
                if "mp4" in video_format.lower():
                    print(f"{filename} has an mp4 format.", file=report_file)
                else:
                    print(f"{filename} does not have an mp4 format.", file=report_file)

                # Find and print the codec_name and audio codec
                for stream in video_info.get("streams", []):
                    if stream.get("index") == 0:
                        video_codec = stream.get("codec_name")

                        if video_codec == "h264":
                            print(f"{filename} has video codec h.264", file=report_file)
                        else:
                            print(f"{filename} does not have video codec h.264", file=report_file)
                    if stream.get("index") == 1:
                        audio_codec = stream.get("codec_name")

                        if audio_codec == "aac":
                            print(f"{filename} has audio codec aac", file=report_file)
                        else:
                            print(f"{filename} does not have audio codec aac", file=report_file)

                # Find and print the frame rate, aspect ratio, resolution, video bit rate, audio bit rate, and audio channel
                for stream in video_info.get("streams", []):
                    if stream.get("index") == 0:
                        avg_frame_rate = stream.get("avg_frame_rate")
                        aspect_ratio = stream.get("display_aspect_ratio")
                        width = stream.get("width")
                        height = stream.get("height")
                        video_bit_rate = stream.get("bit_rate")

                        # Convert bit_rate to Mb/s
                        bit_rate_mbps = round(int(video_bit_rate) / 1000000, 2)

                        # Convert avg_frame_rate to a decimal number
                        numerator, denominator = map(int, avg_frame_rate.split('/'))
                        frame_rate = round(numerator / denominator, 2)

                        if frame_rate == 25.0:
                            print(f"{filename} has a frame rate of 25 FPS", file=report_file)
                        else:
                            print(f"{filename} does not have a frame rate of 25 FPS", file=report_file)

                        if aspect_ratio == "16:9":
                            print(f"{filename} has an aspect ratio of 16:9", file=report_file)
                        else:
                            print(f"{filename} does not have an aspect ratio of 16:9", file=report_file)

                        if width == 640 and height == 360:
                            print(f"{filename} has a resolution of 640 x 360", file=report_file)
                        else:
                            print(f"{filename} does not have a resolution of 640 x 360", file=report_file)

                        if 2.0 < bit_rate_mbps < 5.0:
                            print(f"{filename} has a bit rate between 2 - 5 Mb/s", file=report_file)
                        else:
                            print(f"{filename} does not have a bit rate between 2 - 5 Mb/s", file=report_file)

                    if stream.get("index") == 1:
                        audio_bit_rate = stream.get("bit_rate")
                        audio_channels = stream.get("channel_layout")

                        # Convert audio bit rate into kbps
                        bit_rate_kbps = round(int(audio_bit_rate) / 1000, 2)

                        if bit_rate_kbps < 256:
                            print(f"{filename} has an audio bit rate up to 256 kb/s", file=report_file)
                        else:
                            print(f"{filename} does not have an audio bit rate up to 256 kb/s", file=report_file)

                        if audio_channels == "stereo":
                            print(f"{filename} has a stereo audio channel", file=report_file)
                        else:
                            print(f"{filename} does not have a stereo audio channel", file=report_file)
                            print()

            except subprocess.CalledProcessError as e:
                print(f"Error running ffprobe for {filename}: {e}", file=report_file)

            # Add an extra newline for spacing after each video file
            print(file=report_file)

# Print a message indicating whether the report was generated successfully
try:
    with open("report.txt", "r") as report_file:
        print("Report generated successfully.")
except FileNotFoundError:
    print("Error: Report file not found. There may have been an issue generating the report.")



Report generated successfully.


In [8]:
# Define the video directory, converted directory, and extensions
video_directory = "Exercise3_Films"
converted_directory = "Converted_Films"

# Create the converted directory if it doesn't exist
os.makedirs(converted_directory, exist_ok=True)

# Iterate through each file in the directory
for filename in os.listdir(video_directory):
    video_file_path = os.path.join(video_directory, filename)
    print("Current file name:", filename)

    # Run ffprobe command for the current video file
    ffprobe_command = f'ffprobe -v error -show_entries format:stream -of json "{video_file_path}"'

    try:
        ffprobe_output = subprocess.check_output(ffprobe_command, shell=True)
        # Parse the JSON output
        video_info = json.loads(ffprobe_output)

        # Extract video format information
        video_format = video_info.get("format", {}).get("format_name")

        # Check if all conditions are met
        conditions_met = (
            "mp4" in video_format.lower() and
            all(stream.get("codec_name") == "h264" for stream in video_info.get("streams", []) if stream.get("index") == 0) and
            all(stream.get("codec_name") == "aac" for stream in video_info.get("streams", []) if stream.get("index") == 1) and
            all(int(stream.get("avg_frame_rate").split('/')[0]) / int(stream.get("avg_frame_rate").split('/')[1]) == 25 for stream in video_info.get("streams", []) if stream.get("index") == 0) and
            all(stream.get("display_aspect_ratio") == "16:9" for stream in video_info.get("streams", []) if stream.get("index") == 0) and
            all(stream.get("width") == 640 and stream.get("height") == 360 for stream in video_info.get("streams", []) if stream.get("index") == 0) and
            all(2.0 <= int(stream.get("bit_rate")) / 1000000 <= 5.0 for stream in video_info.get("streams", []) if stream.get("index") == 0) and
            all(int(stream.get("bit_rate")) / 1000 <= 256 for stream in video_info.get("streams", []) if stream.get("index") == 1) and
            all(stream.get("channel_layout") == "stereo" for stream in video_info.get("streams", []) if stream.get("index") == 1)
        )

        # Check if all conditions are met
        if conditions_met:
            print(f"{filename} meets the requirements.")
        else:
            print(f"{filename} does not meet the requirements. Converting...")

            # Define the output file path with a unique identifier in the filename
            converted_file_path = os.path.join(converted_directory, os.path.splitext(filename)[0] + "_formatOK" + ".mp4")

            # Run ffmpeg command to convert the video to meet the requirements
            ffmpeg_command = f'ffmpeg -i "{video_file_path}" -c:v libx264 -c:a aac -strict experimental -r 25 -s 640x360 -aspect 16:9 -b:v 4M -b:a 256k -ac 2 "{converted_file_path}"'
            subprocess.run(ffmpeg_command, shell=True)

            print(f"{filename} has been converted to meet the requirements. Saved at {converted_file_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error running ffprobe for {filename}: {e}")

Current file name: Cosmos_War_of_the_Planets.mp4
Cosmos_War_of_the_Planets.mp4 does not meet the requirements. Converting...
Cosmos_War_of_the_Planets.mp4 has been converted to meet the requirements. Saved at Converted_Films\Cosmos_War_of_the_Planets_formatOK.mp4
Current file name: Last_man_on_earth_1964.mov
Last_man_on_earth_1964.mov does not meet the requirements. Converting...
Last_man_on_earth_1964.mov has been converted to meet the requirements. Saved at Converted_Films\Last_man_on_earth_1964_formatOK.mp4
Current file name: The_Gun_and_the_Pulpit.avi
The_Gun_and_the_Pulpit.avi does not meet the requirements. Converting...
The_Gun_and_the_Pulpit.avi has been converted to meet the requirements. Saved at Converted_Films\The_Gun_and_the_Pulpit_formatOK.mp4
Current file name: The_Hill_Gang_Rides_Again.mp4
The_Hill_Gang_Rides_Again.mp4 does not meet the requirements. Converting...
The_Hill_Gang_Rides_Again.mp4 has been converted to meet the requirements. Saved at Converted_Films\The_Hil

In [11]:
converted_new_directory = "Converted_Films"

# Iterate through each file in the directory
for filename in os.listdir(converted_new_directory):
    video_file_path = os.path.join(converted_new_directory, filename)

    # Skip non-video files or directories
    if not os.path.isfile(video_file_path):
        continue

    # Run ffprobe command for the current video file
    ffprobe_command = f'ffprobe -v error -show_entries format:stream -of json "{video_file_path}"'

    try:
        ffprobe_output = subprocess.check_output(ffprobe_command, shell=True)
        # Parse the JSON output
        video_info = json.loads(ffprobe_output)
        # Extract video format information
        video_format = video_info.get("format", {}).get("format_name")
        
        # Check if the video format contains "mp4"
        if "mp4" in video_format.lower():
            print(f"{filename} has an mp4 format. Video format is: {video_format}")
        else:
            print(f"{filename} does not have an mp4 format. Video format is: {video_format}")
        
        # Find and print the codec_name  and audio codec
        for stream in video_info.get("streams", []):
            if stream.get("index") == 0:
                video_codec = stream.get("codec_name")
                
                if video_codec == "h264":
                    print(f"{filename} has video codec h.264. Video codec is: {video_codec}")
                else:
                    print(f"{filename} does not have video codec h.264. Video codec is: {video_codec}")
            if stream.get("index") == 1:
                audio_codec = stream.get("codec_name")
                
                if audio_codec == "aac":
                    print(f"{filename} has audio codec aac. Audio codec is: {audio_codec}")
                else:
                    print(f"{filename} does not have audio codec aac. Audio codec is: {audio_codec}")
        
        # Find and print the frame rate, aspect ratio, resolution, video bit rate, audio bit rate and audio channel
        for stream in video_info.get("streams", []):
            if stream.get("index") == 0:
                avg_frame_rate = stream.get("avg_frame_rate")
                aspect_ratio = stream.get("display_aspect_ratio")
                width = stream.get("width")
                height = stream.get("height")
                video_bit_rate = stream.get("bit_rate")
                
                # Convert bit_rate to Mb/s
                bit_rate_mbps = round(int(video_bit_rate)/1000000, 2)
                
                if avg_frame_rate == "25/1":
                    print(f"{filename} has a frame rate of 25 FPS. Frame rate is: {avg_frame_rate}")
                else:
                    print(f"{filename} does not have a frame rate of 25 FPS. Frame rate is: {avg_frame_rate}")
                
                if aspect_ratio == "16:9":
                    print(f"{filename} has an aspect ratio of 16:9. Aspect ratio is: {aspect_ratio}")
                else:
                    print(f"{filename} does not have an aspect ratio of 16:9. Aspect ratio is: {aspect_ratio}")
                    
                if width == 640:
                    if height == 360:
                        print(f"{filename} has a resolution of 640 x 360. Resolution is {width} x {height}")
                else:
                    print(f"{filename} does not have a resolution of 640 x 360. Resolution is {width} x {height}")
                    
                if 2.0 <= bit_rate_mbps <= 5.0:
                    print(f"{filename} has a bit rate between 2 - 5 Mb/s. Bit rate is: {bit_rate_mbps}")
                else:
                    print(f"{filename} does not have a bit rate between 2 - 5 Mb/s. Bit rate is: {bit_rate_mbps}")
                
            
            if stream.get("index") == 1:
                audio_bit_rate = stream.get("bit_rate")
                audio_channels = stream.get("channel_layout")
                
                # Convert audio bit rate into kpbs
                bit_rate_kbps = round(int(audio_bit_rate)/1000, 2)
                
                if bit_rate_kbps < 256:
                    print(f"{filename} has an audio bit rate up to 256 kb/s. Audio bit rate is {bit_rate_kbps} kb/s")
                else:
                    print(f"{filename} does not have an audio bit rate up to 256 kb/s. Audio bit rate is {bit_rate_kbps} kb/s")
                
                if audio_channels == "stereo":
                    print(f"{filename} has a stereo audio channel. Audio channel is: {audio_channels}")
                else:
                    print(f"{filename} does not have a stereo audio channel. Audio channel is: {audio_channels}")
                print()

    except subprocess.CalledProcessError as e:
        print(f"Error running ffprobe for {filename}: {e}")

Cosmos_War_of_the_Planets_formatOK.mp4 has an mp4 format. Video format is: mov,mp4,m4a,3gp,3g2,mj2
Cosmos_War_of_the_Planets_formatOK.mp4 has video codec h.264. Video codec is: h264
Cosmos_War_of_the_Planets_formatOK.mp4 has audio codec aac. Audio codec is: aac
Cosmos_War_of_the_Planets_formatOK.mp4 has a frame rate of 25 FPS. Frame rate is: 25/1
Cosmos_War_of_the_Planets_formatOK.mp4 has an aspect ratio of 16:9. Aspect ratio is: 16:9
Cosmos_War_of_the_Planets_formatOK.mp4 has a resolution of 640 x 360. Resolution is 640 x 360
Cosmos_War_of_the_Planets_formatOK.mp4 has a bit rate between 2 - 5 Mb/s. Bit rate is: 3.96
Cosmos_War_of_the_Planets_formatOK.mp4 has an audio bit rate up to 256 kb/s. Audio bit rate is 245.89 kb/s
Cosmos_War_of_the_Planets_formatOK.mp4 has a stereo audio channel. Audio channel is: stereo

Last_man_on_earth_1964_formatOK.mp4 has an mp4 format. Video format is: mov,mp4,m4a,3gp,3g2,mj2
Last_man_on_earth_1964_formatOK.mp4 has video codec h.264. Video codec is: h264